In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from re import M

pd.set_option("display.max_columns", None)

In [3]:
# Get approximate number of available training examples (COINS data)

swan = pd.read_csv('data/SWAN.csv', engine='python')
whodas = pd.read_csv('data/WHODAS.csv', engine='python')
diagnoses = pd.read_csv('data/assessment_data/9994_ConsensusDx_20211130.csv', engine='python')
diagnoses = diagnoses.rename(columns = lambda column: column.split(",")[-1])
# Ignore the extra extra header and drop duplicates
swan = swan.iloc[1:, :].drop_duplicates()
whodas = whodas.iloc[1:, :].drop_duplicates()
diagnoses = diagnoses.iloc[1:, :].drop_duplicates()

In [4]:
merged_swan_diag = pd.merge(swan, diagnoses, on="EID")
merged_whodas_diag = pd.merge(whodas, diagnoses, on="EID")

In [5]:
merged_swan_diag["EID"].count()

3479

In [6]:
merged_swan_diag["EID"].nunique()

3144

In [7]:
merged_whodas_diag["EID"].count()

2695

In [8]:
merged_whodas_diag["EID"].nunique()

2528

In [9]:
# Remove missing diagnoses

merged_whodas_diag = merged_whodas_diag.drop(merged_whodas_diag[
    (merged_whodas_diag["DX_01"] == "No Diagnosis Given: Incomplete Eval")].index)

merged_whodas_diag["EID"].nunique()

2317

In [10]:
# Diagnosis counts (any position)

pd.set_option('display.max_rows', None) 
diag_colunms = ["DX_01", "DX_02", "DX_03", "DX_04", "DX_05", "DX_06", "DX_07", "DX_08", "DX_09", "DX_10"]
merged_whodas_diag[diag_colunms].apply(pd.Series.value_counts).sum(numeric_only=True, axis=1).sort_values(ascending=False)

ADHD-Combined Type                                                             712.0
ADHD-Inattentive Type                                                          706.0
Specific Learning Disorder with Impairment in Reading                          448.0
Generalized Anxiety Disorder                                                   400.0
Autism Spectrum Disorder                                                       390.0
Oppositional Defiant Disorder                                                  330.0
Specific Phobia                                                                305.0
Language Disorder                                                              288.0
Social Anxiety (Social Phobia)                                                 281.0
Specific Learning Disorder with Impairment in Mathematics                      217.0
Enuresis                                                                       213.0
Major Depressive Disorder                                        

In [11]:
# Diagnosis combinations counts

def clean_diag_list(row):
    row_without_duplicates = list({x for x in set(row) if x==x})
    row_without_duplicates.sort()
    return str(row_without_duplicates)

list_of_diagnosis_sets = list(map(clean_diag_list, merged_whodas_diag[diag_colunms].to_numpy()))

counter = Counter(list_of_diagnosis_sets).most_common()
diag_combination_counter_df = pd.DataFrame.from_dict(counter)

pd.set_option('display.max_colwidth', None)

diag_combination_counter_df[diag_combination_counter_df[1] > 1].head(10000) # only display combinations that occure more than once

,0,1
0,['No Diagnosis Given'],164
1,['ADHD-Inattentive Type'],154
2,['ADHD-Combined Type'],141
3,['Specific Learning Disorder with Impairment in Reading'],54
4,"['ADHD-Combined Type', 'Oppositional Defiant Disorder']",43
5,['Major Depressive Disorder'],36
6,['Other Specified Attention-Deficit/Hyperactivity Disorder'],35
7,"['ADHD-Combined Type', 'Autism Spectrum Disorder']",34
8,"['ADHD-Inattentive Type', 'Autism Spectrum Disorder']",32
9,['Generalized Anxiety Disorder'],25


In [12]:
pd.set_option('display.max_rows', 20) 

## Explore questionnaire distribution
### LORIS data

In [ ]:
# LORIS saved query (all data)

full = pd.read_csv("data/all1.csv", dtype=object)

In [ ]:
# Check which questionnaires are in the rows with unserscores (NDARZZ007YMP_1, NDARAA075AMK_Visit_1)

rows_with_underscore_in_id = full[full["Identifiers"].str.contains("_")]
non_empty_columns = rows_with_underscore_in_id.columns[
    ~rows_with_underscore_in_id.isin(["."]).all()].tolist()
non_empty_questionnaires_in_underscore = set([x.split(",")[0] for x in non_empty_columns])
non_empty_questionnaires_in_underscore

In [ ]:
# Drop rows with underscores and the questionnaires present in rows with underscores 

non_empty_questionnaires_in_underscore.remove("Identifiers")
main = full[~full["Identifiers"].str.contains("_")]
len(main.columns)

In [ ]:
filtered_main = main
for questionnaire in non_empty_questionnaires_in_underscore:
    filtered_main = filtered_main.drop(filtered_main.filter(regex=(questionnaire+",")), axis=1)
    len(filtered_main.columns)

In [ ]:
# Exract ID

filtered_main["ID"] = filtered_main["Identifiers"].str.split(',').str[0].str.split('_').str[0]

# Drop IDs with quotes around them (duplicates)

filtered_main[filtered_main["ID"].str.contains("'")]
filtered_main = filtered_main.drop(filtered_main[filtered_main["ID"].str.contains("'")].index)

In [ ]:
filtered_main["Identifiers"].count()

More unique IDs than in release notes

In [ ]:
# filtered_main["Identifiers"].value_counts() # large output

In [ ]:
filtered_main["ID"].count()

In [ ]:
filtered_main["ID"].nunique()

In [ ]:
# Replace . with NaN

filtered_main = filtered_main.replace(".", np.nan)

In [ ]:
# How many people answered all questionnaires (Data_entry value for every questionnaire is not empty)

data_entry_columns = filtered_main.filter(regex=(",Data_entry"))
data_entry_columns.dropna(how = 'any')

In [ ]:
# Get questionnaires most people answered

pd.set_option('display.max_rows', None)
questionnaire_answer_counts = data_entry_columns.count().sort_values(ascending=False).to_frame()
questionnaire_answer_counts["Ratio"] = questionnaire_answer_counts[0]/filtered_main["ID"].nunique()*100
questionnaire_answer_counts

In [ ]:
filtered_main[filtered_main["ASR,Data_entry"].notna()]["ID"] # Check that all IDs seem valid 

In [ ]:
# List of top mental health quetsionnaires and output variables (consensus diagnosis and impairment)

top_mh_assessments = ['Diagnosis_ClinicianConsensus', 'ARI_S', 'SympChck', 'SCQ', 'ASSQ', 'SDQ', 'ARI_P', 'SWAN', 'SRS', 'CBCL', 'NLES_P', 
'SCARED_P', 'ICU_P', 'PCIAT', 'DTS', 'ESWAN', 'MFQ_P', 'YFAS_C', 'WHODAS_P', 'SDS', 'CIS_P', 'SAS', 'WHODAS_SR', 
'CIS_SR', 'CGAS', 'RBS', 'C3SR', 'SCARED_SR', 'MFQ_SR', 'CCSC', 'YSR', 'CDI_P', 'CDI_SR', 'YFAS', 
'CBCL_Pre', 'SRS_Pre', 'ASR', 'CAARS', 'STAI']

In [ ]:
# get number of people who took all top 1, top 2, top 3, etc. popular assessments

cumul_number_of_examples_list = []
for i in range(1, len(top_mh_assessments)+1):
    columns = [x+",Data_entry" for x in top_mh_assessments[0:i]]
    cumul_number_of_examples = data_entry_columns[columns].notnull().all(axis=1).sum()
    print([x.split(",")[0] for x in columns], ": ", cumul_number_of_examples)
    cumul_number_of_examples_list.append(cumul_number_of_examples)

### COINS data

In [ ]:
# Read relevant coins data (Consensus diagnosis + Questionnnaire Measures of Emotional and Cognitive Status from Assessment_List_Jan2019) + Basic demos and Baratt
# exclude TRF and TRF_Pre for now, to see what to do with multiple entries per person

assessments = ['ConsensusDx', 'Basic_Demos', 'Barratt', 'ASR', 'YSR', 'MFQ_SR', 'SCARED_SR', 'C3SR', 'ARI_S', 'CSSRS', 'CIS_SR', 'WHODAS_SR', 
'ICU_SR', 'PANAS', 'CBCL', 'CBCL_Pre', 'MFQ_P', 'SCARED_P', 'ESWAN', 'SWAN', 'ASSQ', 'SCQ', 'CIS_P', 
'SAS', 'WHODAS_P', 'SDQ', 'RBS', 'SRS', 'SRS_Pre', 'ARI_P', 'ICU_P', 'SympChck']

# Read all assessment files into dataframes

coins_file_names = ['9994_' + x + '_20211130.csv' for x in assessments]

all_coins_dfs = []
for file_name in coins_file_names:
    temp_df = pd.read_csv('data/assessment_data/' + file_name, header=1, engine='python')
    all_coins_dfs.append(temp_df)

In [ ]:
# Check that no assessments have duplicated IDs (will aggregate assessment by ID by taking the first non-null value)

for df in all_coins_dfs:
    print(df.columns[4], ": ", df["ID"].count() == df["ID"].nunique())

Exclude 'TRF', 'TRF_Pre' from the assessment list for now, investigate ConsensusDx

In [ ]:
# Investigate duplicated IDs in ConsensusDx

all_coins_dfs[0]["ID"].count()

In [ ]:
all_coins_dfs[0]["ID"].nunique()

In [ ]:
all_coins_dfs[0] = all_coins_dfs[0].drop_duplicates()
all_coins_dfs[0]["ID"].count()

In [ ]:
# all_coins_dfs[0][all_coins_dfs[0].duplicated(['ID'], keep=False)][0:10] # output suppressed for privacy

Seems like the second line in each pair has more values, will take second line as truth, to investigate further

In [ ]:
all_coins_dfs[0] = all_coins_dfs[0].groupby("ID").last().reset_index()
all_coins_dfs[0]["ID"].count()

In [ ]:
# Concatenate all assessments

all_coins = pd.concat(all_coins_dfs)
del all_coins_dfs

all_coins.count()[:10]

In [ ]:
all_coins["ID"].nunique()

In [ ]:
# Group assessements by person

grouped_coins = all_coins.groupby(by=["ID", "SUB_TYPE", "VISIT", "DAY_LAG"]).first()

In [ ]:
pd.set_option('display.max_rows', 100) 
grouped_coins = grouped_coins.reset_index()
grouped_coins.count()

In [ ]:
# Test grouping on a small subset

ASR = pd.read_csv('data/test/' + coins_file_names[3], header=1)
YSR = pd.read_csv('data/test/' + coins_file_names[4], header=1)
MFQ = pd.read_csv('data/test/' + coins_file_names[5], header=1)
ASR_YSR_MFQ_appended = pd.concat([ASR, YSR, MFQ])

In [ ]:
grouped_test = ASR_YSR_MFQ_appended.groupby(by=["ID", "SUB_TYPE", "VISIT", "DAY_LAG"])
# grouped_test.apply(display) # output suppressed for privacy

In [ ]:
agg_test = grouped_test.first() 
# agg_test # output suppressed for privacy

In [ ]:
# Get questionnaires most people answered

pd.set_option('display.max_rows', None)
assessment_column_names = grouped_coins.columns[grouped_coins.columns.str.contains("_001")]
total_participants = grouped_coins["ID"].nunique()
coins_questionnaire_answer_counts = grouped_coins[assessment_column_names].count().sort_values(ascending=False).to_frame()
coins_questionnaire_answer_counts["Ratio"] = coins_questionnaire_answer_counts[0]/total_participants*100
coins_questionnaire_answer_counts

In [ ]:
# Remove incomplete Dx

grouped_coins = grouped_coins[grouped_coins["ConsensusDx_010"] != "No Diagnosis Given: Incomplete Eval"]
grouped_coins["ConsensusDx_010"].value_counts()[:10]

In [ ]:
# Re-calculate questionnaires most people answered without incomplete diagnoses

coins_questionnaire_answer_counts = grouped_coins[assessment_column_names].count().sort_values(ascending=False).to_frame()
coins_questionnaire_answer_counts["Ratio"] = coins_questionnaire_answer_counts[0]/total_participants*100
coins_questionnaire_answer_counts

In [ ]:
# Get number of people who took all top 1, top 2, top 3, etc. popular assessments

assessments_in_order = coins_questionnaire_answer_counts.index.to_list() # extract assessemnts in order of popularity
cumul_number_of_examples_list = []
for i in range(1, len(assessments_in_order)+1):
    columns = assessments_in_order[0:i] # top i assessments
    cumul_number_of_examples = grouped_coins[columns].notnull().all(axis=1).sum()
    print(columns, ": ", cumul_number_of_examples)
    cumul_number_of_examples_list.append(cumul_number_of_examples)

In [ ]:
# List of most popular assessments until the first one with an age restriction
first_columns_until_SCARED = [x for x in assessments_in_order[:assessments_in_order.index("SCAREDSR_001")]]
first_columns_until_SCARED

In [ ]:
# Get only people who took the most popular assessments until the first one with an age restriction (SCARED)
columns_until_SCARED = []
assessments_until_SCARED = [x.split("_")[0] for x in first_columns_until_SCARED]
for assessment in assessments_until_SCARED:
    columns = [column for column in grouped_coins.columns if(assessment in column)]
    columns_until_SCARED.extend(columns)
    
data_up_to_SCARED = grouped_coins.loc[grouped_coins[first_columns_until_SCARED].dropna(how="any").index][columns_until_SCARED+["ID"]]

# Investigate missing values (up to SCARED)
missing_report_up_to_SCARED = data_up_to_SCARED.isna().sum().to_frame(name="Amount missing")
missing_report_up_to_SCARED["Persentage missing"] = missing_report_up_to_SCARED["Amount missing"]/data_up_to_SCARED["ID"].nunique() * 100
missing_report_up_to_SCARED[
    (~missing_report_up_to_SCARED.index.str.contains("ConsensusDx")) # don't print missing data in dx because it's expected
    &
    (missing_report_up_to_SCARED["Persentage missing"] > 1)
].sort_values(ascending=False, by="Amount missing").style.format(precision=2)

In [ ]:
# Check if missing columns are dependent on each other

import missingno as msno
msno.heatmap(grouped_coins[assessment_column_names])

In [ ]:
# Check if missing columns are dependent on demographics and barratt

data_for_na_report = grouped_coins.copy()
data_for_na_report[assessment_column_names] = data_for_na_report[assessment_column_names].notnull().astype('int')
data_for_na_report["ASRHBN_001"].value_counts()

corr_matrix = data_for_na_report[list(assessment_column_names) + ["BASIC1_004", "BASIC1_005"]].corr().drop(labels=["BASIC1_004", "BASIC1_005"])
corr_matrix[["BASIC1_004", "BASIC1_005"]].style.background_gradient(cmap='coolwarm', vmin=-0.8, vmax=0.8).format(precision=2)

Besides expected results (e.g. "_Pre" assessements missing for older kids): 
* **CSSRS** - more missing assessments for yournger kids
* **SDQ**, **CSC** - more missing assessments for older kids

In [ ]:
corr_matrix = data_for_na_report[list(assessment_column_names) + ["BARRATT1_008", "BARRATT1_009", "BARRATT1_010", 
                                                       "BARRATT1_011", "BARRATT1_012", "BARRATT1_013", "BARRATT1_014"]].corr().drop(labels=["BARRATT1_008", 
                                                                                                                                            "BARRATT1_009", "BARRATT1_010", 
                                                                                                                                            "BARRATT1_011", "BARRATT1_012", 
                                                                                                                                            "BARRATT1_013", "BARRATT1_014"])
corr_matrix[["BARRATT1_008", "BARRATT1_009", "BARRATT1_010",
             "BARRATT1_011", "BARRATT1_012", "BARRATT1_013", "BARRATT1_014"]].style.background_gradient(cmap='coolwarm', vmin=-0.8, vmax=0.8).format(precision=2)


In [ ]:
# How many variables we have in top 19 questionnaires (up to data_up_to_SCARED)
data_up_to_SCARED.columns.str.contains("ConsensusDx")